# DELIVERY COSTS 

## Rationale 
The costs for delivering the goods to the customers turned out greater than expected. The purpose of this notebook is to analyse geographically how this costs are distributed. This should give a clearer picture of what is spent where, so that in the future it will be easier to evaluate alternatives.

## The dataset
The table consists of all the movements in a logistic account. It is structured as debits, for every delivery and periodic fees, and credits, for deposits and reversals.
* date
* movement_id: for deliveries it is an alphanumerical code, for other movements it is just a description
* movement_type: desctription of the reason 
* amount(signed)
* Note: additional notes, for reversals it contains the original movement_id 

In [1]:
import pandas as pd
import matplotlib as plt

raw_delivery_movs = (pd.read_excel('data\\estratticonto.xls', 
                                usecols = ['Data', 'ID Transizione',  
                                           'Tipologia movimento',
                                           'Importo', 'Note']
                               ).rename({
                                'Data':'date',
                                'ID Transizione':'movement_id',
                                'Tipologia movimento':'movement_type',
                                'Importo':'amount'
                                   }, axis='columns'    
                               )
                 )
                        

In [2]:
raw_delivery_movs.head()

,date,movement_id,movement_type,amount,Note
0,17/05/2023,Prima Ricarica,Richiesta credito,99.0000,NaN
1,17/05/2023,ricarica formazione,Richiesta credito,20.0000,NaN
2,17/05/2023,287670I028608,Spedizione,-7.2346,NaN
3,17/05/2023,287670I028609,Spedizione,-7.2346,NaN
4,17/05/2023,287670I028609,Spedizione,7.2346,Accredito annullamento spedizione 287670I028609


### Remove NAs

In [3]:
delivery_movs = raw_delivery_movs.dropna(subset='movement_id')

### Remove reversals
Posting and reversals have the same ID, so I can remove all the rows whose IDs are in the list of reversal movements IDs.  

In [4]:
raw_reversals = delivery_movs.dropna()
reversals = raw_reversals.loc[raw_reversals['Note'].str.contains('nnull'), 'movement_id'].to_numpy()
final_delivery_movs = delivery_movs.loc[~delivery_movs['movement_id'].isin(reversals)]

### Replace sign and categories values with more understandable (english) ones

In [5]:
final_delivery_movs['movement_type'] = final_delivery_movs['movement_type'].replace(
                                                                                  ['Richiesta credito', 'Spedizione', 'Ricontabilizzazione'],
                                                                                  ['deposit','delivery', 'fee']
                                                                                   )

C:\Users\samue\AppData\Local\Temp\ipykernel_45460\383988674.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_delivery_movs['movement_type'] = final_delivery_movs['movement_type'].replace(


### Extract year and month from date

In [6]:
final_delivery_movs['date'] = pd.to_datetime(final_delivery_movs['date'])
final_delivery_movs['year'] = final_delivery_movs['date'].dt.year
final_delivery_movs['month'] = final_delivery_movs['date'].dt.month
final_delivery_movs.head()

C:\Users\samue\AppData\Local\Temp\ipykernel_45460\1351307065.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_delivery_movs['date'] = pd.to_datetime(final_delivery_movs['date'])
C:\Users\samue\AppData\Local\Temp\ipykernel_45460\1351307065.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_delivery_movs['date'] = pd.to_datetime(final_delivery_movs['date'])
C:\Users\samue\AppData\Local\Temp\ipykernel_45460\1351307065.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,date,movement_id,movement_type,amount,Note,year,month
0,2023-05-17,Prima Ricarica,deposit,99.0000,NaN,2023,5
1,2023-05-17,ricarica formazione,deposit,20.0000,NaN,2023,5
6,2023-05-18,287670I028700,delivery,-7.2346,NaN,2023,5
7,2023-07-13,078070444432,delivery,-34.1966,NaN,2023,7
8,2023-09-06,Logistica e Magazzino Agosto 2023,delivery,-245.0600,NaN,2023,9


## Analysis

### Variable costs by recipient region over time

In [7]:
final_delivery_movs[final_delivery_movs['movement_type']=='delivery'].groupby(['year','month'])['amount'].mean()

year  month
2023  5         -7.234600
      7        -34.196600
      9       -245.060000
      10       -11.730237
      11        -8.024409
      12        -8.811563
2024  1         -9.908840
Name: amount, dtype: float64

### Fixed costs

In [8]:
final_delivery_movs[final_delivery_movs['movement_type']=='fee']

,date,movement_id,movement_type,amount,Note,year,month
159,2023-11-08,Magazzino e logistica Ottobre 2023,fee,-111.50,NaN,2023,11
328,2024-01-05,Logistica e magazzino Dicembre 2023,fee,-128.95,NaN,2024,1


In [9]:
final_delivery_movs[final_delivery_movs['movement_type']=='fee']['amount'].sum()

-240.45